<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Preparation" data-toc-modified-id="Data-Preparation-1">Data Preparation</a></span></li><li><span><a href="#Feature-Engineering" data-toc-modified-id="Feature-Engineering-2">Feature Engineering</a></span><ul class="toc-item"><li><span><a href="#New-dataframes" data-toc-modified-id="New-dataframes-2.1">New dataframes</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Side-note---Series" data-toc-modified-id="Side-note---Series-2.1.0.1">Side note - Series</a></span></li></ul></li><li><span><a href="#New-dataframe-based-on-columns-(selecting)" data-toc-modified-id="New-dataframe-based-on-columns-(selecting)-2.1.1">New dataframe based on columns (selecting)</a></span><ul class="toc-item"><li><span><a href="#Select-using-column-names" data-toc-modified-id="Select-using-column-names-2.1.1.1">Select using column names</a></span></li><li><span><a href="#Select-using-.loc[-]" data-toc-modified-id="Select-using-.loc[-]-2.1.1.2">Select using .loc[ ]</a></span></li><li><span><a href="#Select-using-.iloc[-]" data-toc-modified-id="Select-using-.iloc[-]-2.1.1.3">Select using .iloc[ ]</a></span></li></ul></li><li><span><a href="#New-dataframe-based-on-rows-(slicing)" data-toc-modified-id="New-dataframe-based-on-rows-(slicing)-2.1.2">New dataframe based on rows (slicing)</a></span><ul class="toc-item"><li><span><a href="#When-a-column-is-used-as-the-row-index" data-toc-modified-id="When-a-column-is-used-as-the-row-index-2.1.2.1">When a column is used as the row index</a></span></li><li><span><a href="#Slice-using-iloc[-]" data-toc-modified-id="Slice-using-iloc[-]-2.1.2.2">Slice using iloc[ ]</a></span></li></ul></li><li><span><a href="#New-dataframe-based-on-rows-and-columns-(indexing)" data-toc-modified-id="New-dataframe-based-on-rows-and-columns-(indexing)-2.1.3">New dataframe based on rows and columns (indexing)</a></span></li><li><span><a href="#New-dataframe-based-on-row-filtering" data-toc-modified-id="New-dataframe-based-on-row-filtering-2.1.4">New dataframe based on row filtering</a></span></li></ul></li></ul></li><li><span><a href="#New-Columns" data-toc-modified-id="New-Columns-3">New Columns</a></span></li><li><span><a href="#Group-By" data-toc-modified-id="Group-By-4">Group By</a></span><ul class="toc-item"><li><span><a href="#Basics" data-toc-modified-id="Basics-4.1">Basics</a></span></li><li><span><a href="#Create-a-new-dataframe-for-grouped-data" data-toc-modified-id="Create-a-new-dataframe-for-grouped-data-4.2">Create a new dataframe for grouped data</a></span></li></ul></li><li><span><a href="#Using-the-penguin-dataset" data-toc-modified-id="Using-the-penguin-dataset-5">Using the penguin dataset</a></span></li><li><span><a href="#Transforming-Data" data-toc-modified-id="Transforming-Data-6">Transforming Data</a></span></li><li><span><a href="#Filtering-data" data-toc-modified-id="Filtering-data-7">Filtering data</a></span></li><li><span><a href="#Group-by-multiple-categories" data-toc-modified-id="Group-by-multiple-categories-8">Group by multiple categories</a></span></li><li><span><a href="#Group-by-numerical-data-using-.cut()-and-.qcut()" data-toc-modified-id="Group-by-numerical-data-using-.cut()-and-.qcut()-9">Group by numerical data using .cut() and .qcut()</a></span></li><li><span><a href="#Return-to-the-Beer-notebook-and-complete-part-2" data-toc-modified-id="Return-to-the-Beer-notebook-and-complete-part-2-10">Return to the Beer notebook and complete part 2</a></span></li></ul></div>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from numpy.random import randn
pd.set_option("display.precision", 1)
sns.set_style('white')

import warnings

warnings.filterwarnings("ignore")

In [ ]:
df_original = pd.read_csv('https://raw.githubusercontent.com/jimcody2014/python_data/refs/heads/main/diabetes_inspect.csv')

In [ ]:
df_original.head()

# Data Preparation

In [ ]:
df = df_original

In [ ]:
df.drop_duplicates(keep = 'first', inplace = True)

df['encounter_id'] = df['encounter_id'].astype(str)
df['patient_nbr'] = df['patient_nbr'].astype(str)
df['admission_type_id'] = df['admission_type_id'].astype(str)
df['discharge_disposition_id'] = df['discharge_disposition_id'].astype(str)
df['admission_source_id'] = df['admission_source_id'].astype(str)

short_names = {'admission_type_id':'admin_type', # creating a dict of the names to be changed
               'discharge_disposition_id':'discharge_dispo',
               'admission_source_id':'admin_source',
               'num_lab_procedures':'lab_procedures',
               'num_procedures':'procedures'}

df.rename(columns=short_names, inplace=True)
no_age = df[df['age'].isnull()].index
drop_columns = {'medical_specialty','glyburide-metformin','glipizide-metformin',
                'glimepiride-pioglitazone', 'payer_code', 'weight'}
df = df.drop(columns = drop_columns)

df.num_medications.fillna( df.num_medications.mean(),inplace=True )
df['gender'] = df['gender'].replace({'M':'Male', 'Mle':'Male', 'F':'Female'})
df['gender'] = df['gender'].apply(lambda x:x.lower())
df['gender'] = df['gender'].replace({'?':'male', 'unknown/invalid':'male'})   
df = df.loc[df['age']!= 'xyz']
df = df.loc[df.gender != '?']
df = df.drop(no_age, axis = 0) 
df2 = df                           # This is used for new dataframe examples

In [ ]:
data = {
    'Weather': ['Sunny','Sunny','Sunny','Cloudy','Shower','Shower','Sunny'], 
    'Temperature': [78,76,78,68,70,71,82],
    'Wind': [13,28,16,11,26,27,20],
    'Humidity': [30,96,20,22,79,62,10],
}
weather = pd.DataFrame(data, index = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

# Feature Engineering

- New dataframes
    - a subset of columns
    - a subset of rows
- New columns

![subset-dataframe.png](images/subset-dataframe.png)

## New dataframes

In [ ]:
df.columns

### New dataframe based on columns (selecting)

#### Select using column names

In [ ]:
medications = df[['miglitol', 'insulin', 'glipizide']] # Notice that the columns are passed in as a list
print(type(medications))
print()
print(medications.info())

In [ ]:
medications.shape

#### Select using .loc[ ]

In [ ]:
medications2 = df.loc[:,['insulin', 'miglitol']] # Notice that the columns are passed in as a list
print(type(medications2))
print()
print(medications2.info())

#### Select using .iloc[ ]

In [ ]:
medications3 = df.iloc[:,[23,24]] # Notice that the columns are passed in as a list
print(type(medications3))
print()
print(medications3.info())

In [ ]:
medications4 = df.iloc[:,18:25] # Notice that the columns are NOT passed in as a list (because of slicing (the colon))
print(type(medications4))
print()
print(medications4.info())

In [ ]:
del medications
del medications2
del medications3
del medications4

### New dataframe based on rows (slicing)

In [ ]:
weather.head(10)

#### When a column is used as the row index

In [ ]:
rows1 = weather.loc[['Tue', 'Thu', 'Sun']]
rows1

#### Slice using iloc[ ]

In [ ]:
rows2 = df.iloc[[2,4,6],]  # Notice, you do not need the colon
print(type(rows2))
print()
print(rows2.shape)

In [ ]:
rows3 = df.iloc[3:20,]  # Notice, you do not need the colon
print(type(rows3))
print()
print(rows3.shape)

### New dataframe based on rows and columns (indexing)

In [ ]:
rows4 = df.iloc[3:20,5:10] 
print(type(rows4))
print()
print(rows4.shape)

In [ ]:
rows4

In [ ]:
rows5 = df.loc[3:20,['insulin','miglitol']] 
print(type(rows5))
print()
print(rows5.shape)

In [ ]:
df.head(20)

### New dataframe based on row filtering

In [ ]:
df['A1Cresult'] == 'None'

In [ ]:
NoA1C = df[df['A1Cresult'] == 'None'] # If True, put into new dataframe
print(type(NoA1C))
print()
print(NoA1C.shape)

In [ ]:
NoA1C.head()

In [ ]:
NoA1C2 = df[(df['A1Cresult'] == 'None') & (df['time_in_hospital'] > 4)] # Notice parens around test
print(type(NoA1C2))
print()
print(NoA1C2.shape)

In [ ]:
NoA1C3 = df[(df['A1Cresult'] == 'None') | (df['time_in_hospital'] > 4)] # Notice parens around test
print(type(NoA1C3))
print()
print(NoA1C3.shape)

In [ ]:
NoA1C4 = df[df['lab_procedures'].between(11,25)] 
print(type(NoA1C4))
print()
print(NoA1C4.shape)
print()
print(NoA1C4.head())

# New Columns

In [ ]:
df.info()

In [ ]:
df['new1'] = df.procedures+1

In [ ]:
df.head()

apply() is a dataframe method that replaces loops.  It takes a function as input and applies it to all rows of the dataframe.

In [ ]:
df['new2'] = df['gender'].apply(lambda x:x.upper())

In [ ]:
df['new3'] = df['procedures'].apply(lambda x:x*2)

In [ ]:
# The Row object is a read-only dictionary-like structure which contains the cell values for a particular row. 
def do_math(row):
    return row['procedures'] + row['lab_procedures']

In [ ]:
df['new4'] = df.apply(do_math, axis=1)

In [ ]:
df['new5'] = df.procedures + df.lab_procedures

In [ ]:
df.head()

# Group By

## Basics

Information we might be interested in:
- For each gender, what is the mean for every numeric column? What are they by age group?
- For each gender, what is the mean value of procedures?
- How many rows are there in each group when the grouping is by gender?
- By gender, what are the min, max and median values for number of medications(individualy)?

In [ ]:
df.columns

In [ ]:
# This will create an error message because it is trying to calculate the mean of strings as well as numbers

# df.groupby('gender').mean()

In [ ]:
df.groupby('gender').mean(numeric_only=True)

In [ ]:
df.groupby('gender').procedures.mean() # Requesting the mean of one column

# This code uses attribute-style access (df.procedures) to select the procedures column from the DataFrame.
# After grouping by gender, it calculates the mean of the procedures column.
# It returns a Pandas Series where the index is gender and the values are the mean of the procedures column for each group.

In [ ]:
# Alternative coding

df.groupby('gender')[['procedures']].mean()

# This code uses bracket notation ([['procedures']]) to select the procedures column as a DataFrame (not a Series).
# After grouping by gender, it calculates the mean for the selected column.
# It returns a Pandas DataFrame where the index is gender and the column is procedures with the mean values.

In [ ]:
df.groupby('gender').size()  # .size() provides # of rows in each group

In [ ]:
df.groupby('gender').procedures.min()

In [ ]:
df.groupby('gender').procedures.max()

In [ ]:
df.groupby('gender').procedures.median()

In [ ]:
df.groupby('gender').procedures.count()

### Quick exercise - 1 min

Rewrite the code to return the min, max, median and count of procedures using the bracket notation.

In [ ]:
# 1 - df.groupby('gender').mean(numeric_only=True)    mean for all numeric columns
# 2 - df.groupby('gender').procedures.min()      mean for a specific column
# 3 -            NEXT                                 multiple aggregations for a column(s)

# there is a function called .agg() and it allows specifiying multiple aggregation functions at once

x = df.groupby('gender').procedures.agg(['max', 'min', 'count', 'median', 'mean'])
x

In [ ]:
# with custom column name
df.groupby('gender').procedures.agg(most=('max'),least=('min'),total=('count'),middle=('median'),average=('mean'))

## A few variations

In [ ]:
# This code calculated the mean for all numeric columns

# The line below calculates the mean for specific columns

df.groupby('gender')[['procedures', 'num_medications']].mean()

In [ ]:
# Multiple columns with different aggregations

df.groupby('gender').agg({
    'procedures': 'mean',               # Average of 'procedures'
    'num_medications': 'median',        # Median of 'num medications'
    'time_in_hospital': 'sum'           # Total time in hospital
})


In [ ]:
# Multiple columns each with multiple  aggregations

df.groupby('gender').agg({
    'procedures': ['mean', 'sum'],  # Mean and sum of 'procedures'
    'num_medications': ['min', 'max'],        # Min and max of 'num medications'
})

In [ ]:
# Calculate thge mean and median for all numeric columns

df.groupby('gender').agg({col: ['mean', 'median'] for col in df.select_dtypes('number').columns})

### Key Takeaways:
- Use agg() with dictionaries to apply different aggregation functions to different columns.
- You can use a list of functions to apply multiple operations on a single column.
- For flexibility, combine groupby() with column selection ([['columns']]) when working with specific subsets of data.

## Create a new dataframe for grouped data

In [ ]:
x = df.groupby('A1Cresult')  #### Notice no method on the end
x.groups





In [ ]:
GT7 = x.get_group('>7')
GT7.head()

In [ ]:
x.first()  # first row in each group

In [ ]:
x.last()  # last row in each group

In [ ]:
print(type(x))
print(type(GT7))

In [ ]:
x.mean(numeric_only=True) 

# Using the penguin dataset

It has a wider range of values

In [ ]:
df2 = sns.load_dataset('penguins')
df2.head()

In [ ]:
df2.shape

In [ ]:
# with custom column name
df2.groupby('sex').body_mass_g.agg(
    sex_max=('max'),
    sex_min=('min'),
)

#df.groupby('sex').body_mass_g.agg(sex_max=('max'),sex_min=('min'))

In [ ]:
# Custom aggregation function
def categorize(x):
    m = x.mean()
    return True if m > 4000 else False

df2.groupby('sex').body_mass_g.agg(['max', 'mean', categorize])

In [ ]:
# Use lambda
df2.groupby('sex').body_mass_g.agg(
    ['max', 'mean', lambda x: True if x.mean() > 4000 else False]
)

In [ ]:
# REMINDER


df2.groupby('sex').mean(numeric_only=True)

In [ ]:
# df2.groupby('sex', numeric_only=True).agg(['mean', 'median'])

In [ ]:
df2.groupby('sex').agg({col: ['mean', 'median'] for col in df2.select_dtypes('number').columns})

# Transforming Data

In [ ]:
# A lambda expression for Standardization.  Setting the mean to zero
standardization = lambda x: (x - x.mean()) / x.std()

In [ ]:
g = df2.groupby('sex').body_mass_g.transform(standardization) # apply is used more when the shape of the data frame will change

In [ ]:
h = df2.groupby('sex').body_mass_g.apply(standardization)

In [ ]:
g.shape

In [ ]:
h.shape

# Filtering data

In [ ]:
# How many rows fall into each island group?
df2.groupby('island').size()

In [ ]:
# filter data to return all islands that have at least 100 observations.
df2.groupby('island').filter(lambda x: len(x) >= 100)

# Group by multiple categories

In [ ]:
# Creating a df that is a subset of penguins

small = df2.loc[:, ['species', 'island', 'bill_depth_mm', 'bill_length_mm']]
small

In [ ]:
# Grouping by multiple categories

small.groupby(['species', 'island']).mean()

In [ ]:
df2.groupby(['species', 'island']).mean(numeric_only=True)

In [ ]:
# Group by multi column
df2_groupby_multi = small.groupby(['species', 'island']).mean()
df2_groupby_multi

In [ ]:
df2_groupby_multi.reset_index()

In [ ]:
# A better way is to set as_index=False
small.groupby(['species', 'island'], as_index=False).mean()

In [ ]:
small.groupby('species').mean(numeric_only=True).plot(kind='bar')  # This is actually a pandas plot

# Group by numerical data using .cut() and .qcut()

In [ ]:
df2['mass_group'] = pd.cut(df2['body_mass_g'], 
                          bins=[0, 3000, 4000, 5000, 10000],
                          labels=('small', 'medium','large', 'wow'))
df2.head()

In [ ]:
df2.groupby(pd.cut(df2['body_mass_g'], 
                          bins=[0, 3000, 4000, 5000, 10000],
                          labels=('small', 'medium','large', 'wow'))).mean(numeric_only=True)

In [ ]:
df2.groupby(pd.qcut(df2["body_mass_g"],4, duplicates="drop")).mean(numeric_only = True)

In [ ]:
# Just a note....

In [ ]:
df2.groupby(['species', 'island']).bill_length_mm.sum().reset_index()

In [ ]:
# A different way to write the same code
df2.groupby(['species', 'island'])['bill_length_mm'].sum().reset_index()